In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

MAX_WIDTH = 1280

def display_image(image, title="Image"):
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title(title)
    plt.axis('off')
    plt.show()

def resize_image(image, max_width):
    aspect_ratio = image.shape[0] / image.shape[1]
    new_width = max_width
    new_height = int(aspect_ratio * new_width)
    return cv2.resize(image, (new_width, new_height))

def clean_mask(mask):
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (7, 7))
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    return mask

def create_color_mask(image):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    color_ranges = [
        ([35, 50, 50], [85, 255, 255]),  # Green
        ([0, 50, 50], [10, 255, 255]),  # Red (lower range)
        ([170, 50, 50], [180, 255, 255]),  # Red (upper range)
        ([20, 50, 50], [30, 255, 255]),  # Yellow
        ([100, 50, 50], [130, 255, 255])  # Blue
    ]

    combined_mask = np.zeros(hsv.shape[:2], dtype="uint8")
    for lower, upper in color_ranges:
        mask = cv2.inRange(hsv, np.array(lower), np.array(upper))
        combined_mask = cv2.bitwise_or(combined_mask, mask)

    return clean_mask(combined_mask)

def find_contours_from_mask(mask, min_area=2000):
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return [cnt for cnt in contours if cv2.contourArea(cnt) > min_area]

def warp_card(image, contour, min_area=10000):
    rect = cv2.minAreaRect(contour)
    box = cv2.boxPoints(rect)
    #box = np.int32(box)

    width, height = map(int, rect[1])
    if width * height < min_area:
        return None

    src_pts = np.array(box, dtype="float32")
    dst_pts = np.array([[0, 0], [width-1, 0], [width-1, height-1], [0, height-1]], dtype="float32")
    M = cv2.getPerspectiveTransform(src_pts, dst_pts)
    warped_card = cv2.warpPerspective(image, M, (width, height))

    if height > width:
        warped_card = cv2.rotate(warped_card, cv2.ROTATE_90_CLOCKWISE)

    return warped_card

def draw_bounding_boxes(image, contours, min_area=10000):
    image_with_boxes = image.copy()
    cards = []

    for i, contour in enumerate(contours):
        warped_card = warp_card(image, contour, min_area)
        if warped_card is not None:
            cards.append(warped_card)
            cv2.drawContours(image_with_boxes, [cv2.boxPoints(cv2.minAreaRect(contour)).astype(int)], 0, (0, 255, 0), 2)

    display_image(image_with_boxes, "Bounding Boxes on Original Image")
    return cards

if __name__ == "__main__":
    image = cv2.imread("pictures/uno1.jpeg")
    image = resize_image(image, MAX_WIDTH)
    display_image(image, "Resized Image")

    mask = create_color_mask(image)
    display_image(cv2.bitwise_and(image, image, mask=mask), "Masked Image")
    contours = find_contours_from_mask(mask)

    contoured_image = image.copy()
    cv2.drawContours(contoured_image, contours, -1, (0, 255, 0), 3)
    display_image(contoured_image, "Contours on Original Image")

    cards = draw_bounding_boxes(image, contours)
    for i, card in enumerate(cards):
        display_image(card, f"Warped Card {i+1}")